In [ ]:
from arcgis.gis import GIS

In [ ]:
import geopandas

In [ ]:
import os

In [ ]:
import pandas as pd
import json

### Enter username and password for ArcGIS online

In [ ]:
username = ""

In [ ]:
password = ""

In [ ]:
gis = GIS("https://www.arcgis.com", username, password)

In [ ]:
# -*- coding: utf-8 -*-
"""
Convert Geospatial Data to TopoJSON
- query data from SQL Server

@author: Jeff Yen
"""


import os
import pyodbc
import pandas as pd
import geopandas as gpd
from shapely import wkt
import topojson as tp #topojson: https://mattijn.github.io/topojson/

# Set workspace to the script directory
os.chdir(os.path.dirname(os.path.abspath('__file__')))

# Functions
def get_gis_layer(sql):
    """
    # Query a GIS layer from SQL Server (GeoDepot)
    """
    #establish SQL connection
    conn = pyodbc.connect("Driver={SQL Server};"
                          "Server=sql2014b8;" # TODO: specify server
                          "Database=GeoDepot;" # TODO: specify database
                          "Trusted_Connection=yes;")
    #run query
    resultDF = pd.read_sql_query(sql, conn)
    
    #convert to GeoDF
    resultDF["geometry"] = [wkt.loads(x) for x in resultDF["geometry"]]
    resultDF = gpd.GeoDataFrame(resultDF, crs="EPSG:2230", geometry="geometry")
    
    #drop original shape column
    resultDF = resultDF.drop('Shape', axis=1)
    
    return resultDF

def convert_to_TopoJSON(input_data, outname):
    """
    # Convert a GIS layer to TopoJSON
    # and export attribute table
    """
    #if input projection is not WGS84
    #then do re-projection
    if input_data.crs != "EPSG:4326":
        input_data = input_data.to_crs("EPSG:4326")
    
    #export geography to topojson
    resultTP = tp.Topology(input_data, prequantize=False).to_json()
    Output_object = open(r"..\output\{}.json".format(outname),"w")
    Output_object.write(resultTP)

    #export attribute table as csv (exclude geometry)
    input_data.iloc[:,:-1].to_csv(r'..\output\{}.csv'.format(outname), index = False)
    
    return resultTP


In [ ]:
lyr_name = 'MGRA13PT' #TODO: specify table name

sql = ("SELECT *"
       ", [Shape].ToString() AS [geometry]"
       "FROM [GeoDepot].[gis].[{}]".format(lyr_name)
       )

# Get GIS layer
gisLYR = get_gis_layer(sql)

# Convert GIS layer to TopoJSON and export attribute table
# topoJSON = convert_to_TopoJSON(gisLYR, lyr_name)


In [ ]:
gisLYR

In [ ]:
mgra_map = gis.map("San Diego", zoomlevel=16)

In [ ]:
mgra_map

In [ ]:
# mgra_map.add_layer(gisLYR)
mgra_map.add_layer(gisLYR, {"field_name":"MGRA" })

### find MGRA feature layer   (to-do: remove the "[4]")

In [ ]:
mgra13 = gis.content.search("MGRA", item_type="Feature Layer", )[4]

In [ ]:
mgra13

### map the MGRAs

In [ ]:
mgra_map = gis.map("San Diego", zoomlevel=16)

In [ ]:
# mgra_map.add_layer(mgra13, {"field_name":"MGRA" })

In [ ]:
mgra_map.add_layer(mgra13)

In [ ]:
mgra_map